# FastDDS v2.8.2 数据流程代码解析

本篇文章基于FastDDS v2.8.2版本，官方的publisher and subscriber示例程序，来梳理FastDDS源码实现中的数据收发流程。

官方的publisher and subscriber示例程序链接：
[Writing a simple C++ publisher and subscriber application](https://fast-dds.docs.eprosima.com/en/v2.8.2/fastdds/getting_started/simple_app/simple_app.html)  
<https://fast-dds.docs.eprosima.com/en/v2.8.2/fastdds/getting_started/simple_app/simple_app.html>

![Publisher的静态类图](publisher_class.drawio.png)